In [1]:
# Librerias a utilizar
import requests
import json
from IPython.display import JSON
from keys import *

In [ ]:
# Realizamos la peticion a la API
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/MMMX/2025-09-01T08:00/2025-09-01T20:00"

# Se asignan los parametros de la peticion
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
# Verificamos si la peticion fue exitosa

if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [ ]:
# Mostrar la data en formato JSON

flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [ ]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

# Extraemos la informacion relevante de cada vuelo
def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

# Concatenamos la informacion de todos los vuelos en un solo DataFrame
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,UA 1024,Houston,United,Boeing 737-700,2025-09-01 14:35Z,2025-09-01 08:35-06:00
0,AM 905,Monterrey,Aeromexico,Boeing 737-900,2025-09-01 14:30Z,2025-09-01 08:30-06:00
0,AM 505,Cancún,Aeromexico,Boeing 737-800,2025-09-01 14:30Z,2025-09-01 08:30-06:00
0,AM 205,Guadalajara,Aeromexico,Boeing 737,2025-09-01 14:35Z,2025-09-01 08:35-06:00
0,VB 1355,Monterrey,VivaAerobus,Airbus A321 NEO,2025-09-01 14:40Z,2025-09-01 08:40-06:00
